# Metrics for RecSys 2018
Implementation for validation purposes. The official rules can be found [here](https://recsys-challenge.spotify.com/rules).

In [1]:
import numpy as np

## R-Precision
R-precision is the number of retrieved relevant tracks divided by the number of known relevant tracks (i.e., the number of withheld tracks):

$$ \text{R-precision} = \frac{∣G \bigcap R_{1:|G|}∣}{|G|} $$
The metric is averaged across all playlists in the challenge set. This metric rewards total number of retrieved relevant tracks (regardless of order).

In [2]:
ground_truth = ['1', '2', '3', '5', '8', '99']
prediction = ['5', '8', '13', '3']

In [3]:
def r_precision(ground_truth, prediction):
    """
    R-precision is the number of retrieved relevant tracks divided by the number of known relevant tracks.
    
    Returns:
    ------------
    relevant_tracks: list of all relevant tracks in order of appearance in prediction set
    r-precision metric: float measure of r-precision
    """
    relevant_tracks = []
    for idx, track in enumerate(prediction):
        if track in ground_truth and idx < len(ground_truth):
            relevant_tracks.append(track)
    return relevant_tracks, (len(relevant_tracks) / len(ground_truth))

In [4]:
r_precision(ground_truth, prediction)

(['5', '8', '3'], 0.5)

In [10]:
prediction_two = ['5', '8', '13', '3', '99', '87', '2', '150']

In [11]:
r_precision(ground_truth, prediction_two)

(['5', '8', '3', '99'], 0.6666666666666666)

In [12]:
def r_precision_two(ground_truth, prediction):
    rel_set = set(prediction[:len(ground_truth)]).intersection(set(ground_truth))
    return rel_set, len(rel_set) / len(ground_truth)

In [13]:
r_precision_two(ground_truth, prediction)

({'3', '5', '8'}, 0.5)

In [14]:
r_precision_two(ground_truth, prediction_two)

({'3', '5', '8', '99'}, 0.6666666666666666)

## Normalized discounted cumulative gain (NDCG)
Discounted cumulative gain (DCG) measures the ranking quality of the recommended tracks, increasing when relevant tracks are placed higher in the list. Normalized DCG (NDCG) is determined by calculating the DCG and dividing it by the ideal DCG in which the recommended tracks are perfectly ranked:

$$ DCG = rel_{1} + \sum_{i=2}^{|R|} \frac{rel_{i}}{log_{2}i} $$

The ideal DCG or IDCG is, on our case, equal to:

$$ IDCG = 1 + \sum_{i=2}^{|G \bigcap R|} \frac{1}{log_{2}i}$$ 

If the size of the set intersection of G and R, is empty, then the DCG is equal to 0. The NDCG metric is now calculated as:

$$ NDCG = \frac{DCG}{IDCG} $$

In [15]:
def get_relevance(ground_truth, item):
    """
    Returns relevance measure for playlist predictions.
    
    Returns:
    ------------
    relevance: 1 if track is in ground_truth, 0 otherwise
    """
    if item in ground_truth:
        return 1
    return 0

In [16]:
def dcg(ground_truth, prediction):
    """
    Discounted cumulative gain (DCG) measures the ranking quality of the recommended tracks.
    DCG increases when relevant tracks are placed higher in the list. 
    
    Returns:
    ------------
    relevance: float representing the relevance metric for a given playlist prediction
    """
    relevance = None
    for idx, track in enumerate(prediction):
        if not relevance:
            relevance = get_relevance(ground_truth, track)
        else:
            relevance += get_relevance(ground_truth, track) / np.log2(idx + 1)
    return relevance

In [17]:
def idcg(ground_truth, prediction):
    """
    Maximum measure for a prediction set
    if all predictions were relevant.
    """
    relevance = None
    idx = 0
    for track in prediction:
        idx += 1
        if not relevance:
            relevance = 1
        else:
            relevance += 1 / np.log2(idx)
    return relevance

In [18]:
ground_truth, prediction

(['1', '2', '3', '5', '8', '99'], ['5', '8', '13', '3'])

In [19]:
# 2.5
dcg(ground_truth, prediction)

2.5

In [20]:
# 3.1309...
idcg(ground_truth, prediction)

3.1309297535714578

In [21]:
def ndcg(ground_truth, prediction):
    """
    Normalized discounted cumulative gain (NDCG). 
    
    Returns:
    ------------
    NDCG: float - discounted cumulative gain given the ideal discounted cumulative gain
    
    """
    return dcg(ground_truth, prediction) / idcg(ground_truth, prediction)

In [22]:
# 0.7984...
ndcg(ground_truth, prediction)

0.7984848580994974

## Recommended Song Clicks
Recommended Songs is a Spotify feature that, given a set of tracks in a playlist, recommends 10 tracks to add to the playlist. The list can be refreshed to produce 10 more tracks. Recommended Songs clicks is the number of refreshes needed before a relevant track is encountered. It is calculated as follows:

$$ \text{clicks} = 	\lfloor \frac{\text{arg min}_{i} \{R_{i}: R_{i} \in G \} - 1}{10} \rfloor$$

If the metric does not exist (i.e. if there is no relevant track in (R), a value of 51 is picked (which is 1 + the maximum number of clicks possible).

In [23]:
def rsc(ground_truth, prediction):
    """
    Recommended Songs is a Spotify feature that, given a set of tracks in a playlist, 
    recommends 10 tracks to add to the playlist. 
    The list can be refreshed to produce 10 more tracks. 
    Recommended Songs clicks is the number of refreshes 
    needed before a relevant track is encountered
    
    Returns:
    ------------
    counter: amount of clicks needed for the first relevant song to appear
    """
    counter = 0
    for idx, track in enumerate(prediction):
        if idx % 10 == 0:
            counter += 1
        if track in ground_truth:
            return counter
    return counter + 1

In [24]:
ground_truth_rsc_one = [1]
ground_truth_rsc_two = [499]
ground_truth_rsc_three = [500]
prediction_rsc_one = range(500)

In [25]:
# 1
rsc(ground_truth_rsc_one, prediction_rsc_one)

1

In [26]:
# 50
rsc(ground_truth_rsc_two, prediction_rsc_one)

50

In [27]:
# 51
rsc(ground_truth_rsc_three, prediction_rsc_one)

51